In [ ]:
from IPython.core.debugger import prompt
from autorag.data.legacy.corpus import llama_text_node_to_parquet
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index
from autorag.utils import cast_corpus_dataset
from gitdb.fun import chunk_size
from llama_index.legacy.llms import OpenAI


In [ ]:
from autorag.utils.preprocess import cast_corpus_dataset 

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter

documents = SimpleDirectoryReader("../data/genesis/", recursive=True).load_data()

In [ ]:
from autorag.data.legacy.corpus.llama_index import llama_text_node_to_parquet
nodes = TokenTextSplitter().get_nodes_from_documents(documents=documents, chunk_size=256, chunk_overlap=64)
corpus_df = llama_text_node_to_parquet(nodes)
corpus_df = cast_corpus_dataset(corpus_df)
corpus_df.to_parquet('../data/test_dataset/corpus_new.parquet')

# 질문생성

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
prompt = """다음을 참고로 제네시스 차량 매뉴얼에 관한 내용을 만들었습니다.
매뉴얼을 보고 할 만한 질문을 만드세요.
반드시 제네시스 차량과 관련한 질문이어야 합니다.
만약 주어진 매뉴얼 내용이 제네시스 차량과 관련되지 않았다면,
'제네시스 차량과 관련 없었습니다.'라고 질문을 만드세요.

매뉴얼:
{{text}}

생성할 질문 개수: {{num_questions}}

예시:
[Q]: 제네시스 차량의 타이어 공기압은 얼마인가요?
[A]: 제네시스 차량의 타이어 공기압은 35 PSI입니다.

제네시스 차량과 관련이 없는 매뉴얼의 경우 예시:
[Q]: 제네시스 차량과 관련 없었습니다.
[A]: 제네시스 차량과 관련 없었습니다.
"""

In [7]:
import pandas as pd
from llama_index.llms.openai import OpenAI
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index

corpus_df = pd.read_parquet('../data/test_dataset/corpus_new.parquet', engine='pyarrow')
llm = OpenAI(model='gpt-4o-mini', temperature=0.5)

qa_df = make_single_content_qa(corpus_df=corpus_df, content_size=5, qa_creation_func=generate_qa_llama_index,
                               llm=llm, prompt=prompt, question_num_per_content=2)

0it [00:00, ?it/s]

[12/04/24 22:55:10] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=460521;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=962981;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/04/24 22:55:11] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=627588;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=941633;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/04/24 22:55:12] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=816707;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=254491;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/04/24 22:55:16] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=902495;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=577005;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/04/24 22:55:19] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=123474;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=588368;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

1it [00:11, 11.26s/it]


In [9]:
qa_df.to_parquet('../data/test_dataset/corpus_new_test.parquet')

In [ ]:
import 




